In [6]:
from tensorflow.keras.models import load_model
from keras_bert import get_custom_objects
path = './'

In [7]:
model = load_model(path + 'models/bert_model.h5', custom_objects=get_custom_objects())

c:\Users\dell 3400\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [8]:
import pandas as pd
data = pd.read_csv(path + 'test1.csv', encoding='utf-8')

review_data = data['review'].tolist()
label_data = data['label'].tolist()

In [9]:
# print(data['label'].isna().sum())
data = data.dropna(subset=['label'])

label_data = data['label'].astype(int).tolist()
# print(label_data)

In [10]:
from vncorenlp import VnCoreNLP
tokenizer = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [11]:
import pickle
replace_list = pickle.load(open(path+'vncorenlp/replace_list.pkl','rb'))

In [13]:
import re
from gensim.utils import simple_preprocess
import pandas as pd
from nltk import flatten

In [14]:
VN_CHARS_LOWER = u'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđð'
VN_CHARS_UPPER = u'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸÐĐ'
VN_CHARS = VN_CHARS_LOWER + VN_CHARS_UPPER
def no_marks(s):
    __INTAB = [ch for ch in VN_CHARS]
    __OUTTAB = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"*2
    __OUTTAB += "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"*2
    __r = re.compile("|".join(__INTAB))
    __replaces_dict = dict(zip(__INTAB, __OUTTAB))
    result = __r.sub(lambda m: __replaces_dict[m.group(0)], s)
    return result

In [16]:
stopwords = [
    "anh", "chị", "bạn", "mình", "tôi", "ta", "ông", "họ", "em", "nó",
    "chúng_ta", "chúng_tôi", "trên", "dưới", "trong", "ngoài", "giữa",
    "của", "với", "tại", "đến", "qua", "vào", "bên", "và", "nhưng",
    "hoặc", "mà", "vì", "bởi", "tuy_nhiên", "nếu", "rồi", "sau",
    "sau_đó", "do", "tất_cả", "mỗi", "rất", "nhiều", "ít", "vài",
    "hơn", "hết", "cả", "tất", "bây_giờ", "hôm_nay", "ngày", "đêm",
    "trước", "hiện", "lúc", "khi", "đã", "vừa", "cái", "này", "kia",
    "gì", "điều", "việc", "vậy", "thế", "là", "có", "được", "sẽ",
    "làm", "như", "sao", "tại_sao", "thế_nào", "như_vậy", "vậy_nên",
    "vậy_mà", "vậy_thì", "phải", "đấy", "đây"
]

def preprocess(data):
    token = []
    for text in data:
        check = re.search(r'([a-z])\1+',text)
        if check:
          if len(check.group())>2:
            text = re.sub(r'([a-z])\1+', lambda m: m.group(1), text, flags=re.IGNORECASE) #remove các ký tự kéo dài như hayyy,ngonnnn...

        text = text.strip() #loại dấu cách đầu câu
        text = text.lower() #chuyển tất cả thành chữ thường

        text = re.sub('< a class.+</a>',' ',text)

        for k, v in replace_list.items():       #replace các từ có trong replace_list
          text = text.replace(k, v)

        text = re.sub(r'_',' ',text)

        text = ' '.join(i for i in flatten(tokenizer.tokenize(text)))             #gán từ ghép

        tokens = simple_preprocess(text)
        filtered_tokens = [word for word in tokens if word not in stopwords]
        text = ' '.join(filtered_tokens)

        token.append(text)
    return token

In [17]:
review_data = [str(review) for review in review_data if isinstance(review, str) or pd.notna(review)]
data_test = preprocess(review_data)

In [18]:
import os
import codecs
import numpy as np
import pandas as pd
import tensorflow as tf

In [19]:
token_dict = {}
with codecs.open(path+'vncorenlp/vocab.txt', 'rb','utf-8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [25]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict,cased=True)
SEQ_LEN = 128

In [26]:
from sklearn.model_selection import train_test_split
def load_data(data, sentiments):
    global tokenizer
    indices = []
    for text in data:
        ids, segments = tokenizer.encode(text, max_len=SEQ_LEN)
        indices.append(ids)

    indices = np.array(indices)  # Chuyển đổi thành numpy array
    return [indices, np.zeros_like(indices)], np.array(sentiments)

In [27]:
X_test,Y_test = load_data(data_test,label_data)

In [28]:
y_pred = np.round(model.predict(X_test))

32/32 [==============================] - 296s 9s/step
